In [28]:
import duckdb
import pandas as pd, h3

In [29]:
con = duckdb.connect("ais.duckdb")

In [30]:
# df = con.execute("""
#     SELECT *
#     FROM 'output/mmsi_246714000.parquet'
#     WHERE LATITUDE IS NOT NULL
#         AND LONGITUDE IS NOT NULL
#         AND SPEED IS NOT NULL
#         AND HEADING IS NOT NULL
#         AND COURSE IS NOT NULL
#     USING SAMPLE 100;
# """).fetchdf()

df = con.execute("""
    SELECT *
    FROM 'output/mmsi_246714000.parquet'
    WHERE LATITUDE IS NOT NULL
        AND LONGITUDE IS NOT NULL
        AND SPEED IS NOT NULL
        AND HEADING IS NOT NULL
        AND COURSE IS NOT NULL;
""").fetchdf()

print(df.head())

            TIMESTAMP       MMSI  LATITUDE   LONGITUDE  SPEED  HEADING  COURSE
0 2023-02-01 00:00:20  246714000  1.330283  104.075005   10.6    315.0   315.8
1 2023-02-01 00:00:30  246714000  1.330635  104.074662   10.6    315.0   315.8
2 2023-02-01 00:00:41  246714000  1.331018  104.074282   10.5    315.0   315.9
3 2023-02-01 00:00:51  246714000  1.331365  104.073933   10.5    315.0   315.9
4 2023-02-01 00:00:52  246714000  1.331365  104.073933   10.5    315.0   315.9


In [31]:
res = 10

# random top 1 sample
lat, lon = df.iloc[0][['LATITUDE', 'LONGITUDE']]

# basic usage
cell = h3.latlng_to_cell(lat, lon, res)
center = h3.cell_to_latlng(cell)
boundary = h3.cell_to_boundary(cell)
neighbors = h3.grid_disk(cell, 1)
parent = h3.cell_to_parent(cell, 7)
children = h3.cell_to_children(cell, 9)

print("Point:", lat, lon)
print("Cell:", cell)
print("Center:", center)
print("Boundary:", boundary) 
print("Parent:", parent)
print("Children:", children)

H3ResDomainError: Invalid child resolution 9 for cell 0x8a6526aab75ffff.

In [32]:
df["H3"] = [h3.latlng_to_cell(lat, lon, res) for lat, lon in zip(df["LATITUDE"], df["LONGITUDE"])]
df.head()

,TIMESTAMP,MMSI,LATITUDE,LONGITUDE,SPEED,HEADING,COURSE,H3
0,2023-02-01 00:00:20,246714000,1.330283,104.075005,10.6,315.0,315.8,8a6526aab75ffff
1,2023-02-01 00:00:30,246714000,1.330635,104.074662,10.6,315.0,315.8,8a6526aab667fff
2,2023-02-01 00:00:41,246714000,1.331018,104.074282,10.5,315.0,315.9,8a6526aab667fff
3,2023-02-01 00:00:51,246714000,1.331365,104.073933,10.5,315.0,315.9,8a6526aab667fff
4,2023-02-01 00:00:52,246714000,1.331365,104.073933,10.5,315.0,315.9,8a6526aab667fff


In [33]:
# aggregate per hex
agg = (df.groupby("H3")
         .agg(ship_points=("MMSI","size"),
              unique_mmsi=("MMSI","nunique"),
              mean_speed=("SPEED","mean"))
         .reset_index())

In [34]:
agg.to_parquet("output/mmsi_246714000_h3.parquet", index=False)

In [35]:
# install h3 extension if not already installed
con.execute("INSTALL h3 FROM community;")

In [36]:
con.execute("LOAD h3;")

In [37]:
con.execute("ALTER TABLE feb_ais ADD COLUMN h3 VARCHAR;")
con.execute("ALTER TABLE aug_ais ADD COLUMN h3 VARCHAR;")

CatalogException: Catalog Error: Column with name h3 already exists!

In [ ]:
con.execute("""
    UPDATE feb_ais
    SET h3 = h3_latlng_to_cell(LATITUDE, LONGITUDE, 8)
""")

In [ ]:
con.execute("""
    UPDATE aug_ais
    SET h3 = h3_latlng_to_cell(LATITUDE, LONGITUDE, 8)
""")